# GenAI Task: Create an LinkedIn post generator from Meetup event
2024 Diego Minguzzi

## Task analysis

The idea is creating a simple script that, given:

* the ID of the Meetup event (in the example it is 296972829)
* additional, optional parameters provided by the user

creates the text of the LinkedIn post.

The output is composed of plain text and possibly URLs as text, given that LinkedIn is able to parse and preview them.
In case the script needs API keys to issue requests to OpenAI or Meetup, such keys must be provided as variables.
In case the script does not find a requested API key, it outputs an error message and no output text is generated.

Given that the tool must be simple:

* it is executed locally in the computer of the operator
* no data is stored locally.

The script is written in Python, Linux is the preferred execution environment.
A list of package requirements is provided, so that any user can install and use the script.

In order to distinguish between user input, post content, error messages, a simple GUI, developed using Gradio, is proposed.

## Steps

* given the Meetup event ID, extract the needed data using the Meetup API GraphQL
* also extract the bio of the speaker and the URL to the image, they are provided as output, but not part of the prompt or the generated text.
* with the extracted data compose a prompt for the LLM, making sure that the number of tokens does not exceeds the limits
* show the generated post, or any error message.


In [1]:
%%capture
! pip install gradio
! pip install openai
! pip install tiktoken

In [2]:
import datetime
import dateutil
import gradio as gr
import json
import openai as oai
import os
import pytz
import tiktoken

In [3]:
if os.getenv('OPENAI_API_KEY') is None:
    print('Cannot find OPENAI_API_KEY environment variable.')
    assert False
open_ai_key= os.getenv('OPENAI_API_KEY')
assert(len(open_ai_key)>0)

## Retrieve data from Meetup
Using the GraphQL Playground, available in Meetup, I issued the following request, and I got the reply pasted below.

In [4]:
meetup=request="""
  query {
    event(id: "296972829") {
      title
      eventUrl
      description
      dateTime
      duration
      speakerDetails {
        name
        description
        socialNetworks {
          service
          url
        }
      }
      host {
        name
      }
    }
  }
"""

In [5]:
meetup_json_reply="""
{
  "data": {
    "event": {
      "title": "Azure's AI Symphony: Orchestration, Embeddings & More!",
      "eventUrl": "https://www.meetup.com/genai-gurus/events/296972829",
      "description": "In this presentation, we will delve into the transformative potential of AI in shaping systems design and orchestration. The first half is dedicated to exploring Microsoft's pivotal AI services, such as Azure OpenAI and Cognitive Services, focusing on their practical applications. We'll demonstrate how to effectively prototype using Azure and online tools, including Language Studio, offering hands-on insights into leveraging these platforms for innovative AI solutions. In the latter part of the talk, we will focus on the nuances of machine learning systems design. Here, the discussion revolves around orchestrating these disparate AI services for optimized performance, highlighting key considerations for deploying these systems in production environments. This segment aims to equip attendees with a deeper understanding of the complexities and strategic approaches essential for successful AI implementation in real-world scenarios, underlining the significance of thoughtful orchestration in the ever-evolving landscape of AI technologies.",
      "dateTime": "2023-11-16T18:30+01:00",
      "duration": "PT1H",
      "speakerDetails": {
        "name": "Ioannis Kozaris",
        "description": "Ioannis Kozaris, a Software Engineer specializing in machine learning systems, has been at the forefront of driving innovation in collaborative systems, data storage, and Natural Language Processing systems at Microsoft. He has notably architected ML orchestration systems and intelligent context-aware ML Agents, elevating user experience and optimizing operational efficiency in production applications. Currently, he is part of a task force at Microsoft, dedicated to pioneering applications in Generative AI, setting the course for the future of AI innovations. He is deeply committed to sharing his passion for ML systems with others, aiming to inspire and engage fellow enthusiasts in this evolving field.",
        "socialNetworks": [
          {
            "service": "LINKEDIN",
            "url": "https://www.linkedin.com/in/ioannis-kozaris/"
          }
        ]
      },
      "host": {
        "name": "Carlos Hernandez-Vaquero"
      }
    }
  }
}
"""
reply_struct= json.loads(meetup_json_reply)
event= reply_struct['data']['event']

In [6]:
def request_meetup_event_data( event_id:str):
  reply_struct= json.loads(meetup_json_reply)
  event= reply_struct['data']['event']
  return event

def meetup_json_to_event_data( meetup_json_reply: str):
  reply_struct= json.loads(meetup_json_reply)
  event= reply_struct['data']['event']
  return event

In [7]:
# event= request_meetup_event_data( event_id='296972829')
# event

## Compose the prompt

In [8]:
openai_model='gpt-3.5-turbo'
encoding_name='cl100k_base'
num_max_tokens= 4096
num_max_input_tokens= num_max_tokens
temperature=1.4
gen_ai_guru_description="""Gen AI Guru is an innovative, non-profit organization that stands at the exciting intersection of science and arts, powered by Generative AI.
We recognize the transformative potential of these technologies on the workplace and the broader societal landscape. By joining GenAI Gurus, you don't just join a community; you position yourself in the driving seat of a network of passionate individuals, all committed to shaping the future."""

In [9]:
prompt_template= """
Write an engaging LinkedIn post to share a Meetup event provided to the GenAI Guru community and to attract participants.
DO NOT include hashtags, like #AI, etc. DO NOT include the speaker LinkedIn profile
Describe in details the content of the presentation.
Highlight the opportunity to learn and understand new technologies and be part of a community.

The title of the Meetup is "{title}"

The speaker is {speaker}

The host is {host}

The description of the event is:
{description}

The Meetup will start at {event_time}

---
"""
def compose_prompt(title: str,
                   speaker: str, host:str,
                   description:str,
                   event_time:str):
  data= {'title': title,
         'speaker': speaker,
         'host': host,
         'description':description,
         'event_time':event_time}
  result= prompt_template.format(**data)
  return result

def format_date(date_str: str):
  date_obj = dateutil.parser.parse(date_str)
  date_obj = date_obj.astimezone( pytz.timezone('CET'))
  return date_obj.strftime('%A, %B %d, %Y at %I:%M %p %Z')

In [10]:
test_prompt= compose_prompt(title="""Azure's AI Symphony: Orchestration, Embeddings & More!""",
                            speaker='Ioannis Kozaris',
                            host='Carlos Hernandez-Vaquero',
                            description='Event description',
                            event_time='2023-11-16T18:30+01:00')
# print(test_prompt)

In [11]:
prompt= compose_prompt( title=event['title'],
                        speaker=event['speakerDetails']['name'],
                        host=event['host']['name'],
                        description=event['description'],
                        event_time=event['dateTime'])

In [12]:
tokenizer = tiktoken.get_encoding(encoding_name)

In [13]:
prompt_tokens= tokenizer.encode(prompt)
num_input_tokens= len(prompt_tokens)
print(f'Num input tokens:{num_input_tokens}')
assert( num_input_tokens <= num_max_input_tokens )

Num input tokens:308


## Issue the request to the LLM

In [14]:
client = oai.OpenAI(api_key=open_ai_key)
completion = client.chat.completions.create(model=openai_model,
                                            max_tokens=num_max_tokens,
                                            temperature=temperature,
                                            messages=[{"role": "user", "content": prompt}])

In [15]:
post= completion.choices[0].message.content

In [16]:
date_str= event['dateTime']
post += f'\n\n{format_date(date_str)}'

In [17]:
speaker_name= event['speakerDetails']['name']

if len(event['speakerDetails']['description'])>0:
  speaker_bio= social_platform_url=event['speakerDetails']['description']
  post += f'\n\n{speaker_bio}'

if len(event['speakerDetails']['socialNetworks'])>0:
  social= event['speakerDetails']['socialNetworks'][0]['service']
  social_url= social_platform_url=event['speakerDetails']['socialNetworks'][0]['url']
  post += f'\n{speaker_name} {social} profile: {social_url}'

post += '\n\n'
post += gen_ai_guru_description


In [18]:
print(post)

🚀 Calling all GenAI Gurus! 🚀

Exciting news! We're thrilled to bring you a Meetup event that is sure to ignite your passion for AI and technology. Join us for "Azure's AI Symphony: Orchestration, Embeddings & More!" hosted by the brilliant Carlos Hernandez-Vaquero.

In this one-of-a-kind presentation, Ioannis Kozaris will unravel the transformative power of AI in system design and orchestration. Get ready to explore Microsoft's cutting-edge AI services like Azure OpenAI and Cognitive Services, uncovering their practical applications and innovative solutions. From prototyping with Azure to maximizing online tools like Language Studio, this event promises hands-on insights that will elevate your understanding of AI technology.

But that's not all! Dive deep into the art of machine learning systems design, where Ioannis will provide expert insights on orchestrating AI services for optimum performance. Learn about deploying these systems in real-world environments and gain strategic approa

## Factorize into a Gradio UI interface
Make sure you know the <b>Meetup ID</b> of the event, for example:<br> 
<a target="_blank" href="https://www.meetup.com/it-IT/genai-gurus/events/299901738/">PandasAI Unpacked</a> has Meetup ID = <code>299901738</code><br>
Insert the Meetup ID in the form below.

In [19]:
def generate_meetup_request(input_meetup_event_id:str):
  try:
    stipped_meetup_event_id= input_meetup_event_id.strip()
    if len(stipped_meetup_event_id) <= 0:
      return ( '','Got empty Meetup ID.')    
        
    meetup_request=f"""query {{
        event(id: "{input_meetup_event_id}") {{
          title
          eventUrl
          description
          dateTime
          duration
          speakerDetails {{
            name
            description
            socialNetworks {{
              service
              url
            }}
          }}
          host {{
            name
          }}
        }}
      }}
    """

    return ( meetup_request,'')

  except Exception as exc:
    return ('',prompt,str(exc))

  return ('','Internal error')    

In [20]:
input_meetup_event_id= gr.Textbox(value="296972829", lines=1, max_lines=1, label="Meetup event ID", show_label=True)
output_meetup_request= gr.Textbox(value="", label="Meetup Request", show_label=True)
output_error= gr.Textbox(value="", label="Error", show_label=True)

demo = gr.Interface(fn=generate_meetup_request,
                    inputs=[input_meetup_event_id],
                    outputs=[output_meetup_request,output_error],
                    allow_flagging='never',
                    description='<center><h1>Generate the request for the Meetup Playground</h1></center>')

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.24.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://b6f5b5219af31f7ae7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Paste the Meetup Request to the Meetup Playground.<br>
<a target="_blank" href="https://www.meetup.com/api/playground/#graphQl-playground">Link to the Playground</a> (the Playground is available even while unlogged).<br>
Copy the result, to use it for next step. 

In [21]:
def generate_linkedin_post(input_meetup_response:str, temperature=temperature):
  try:
    prompt=''
    event= meetup_json_to_event_data( input_meetup_response)
    prompt= compose_prompt( title=event['title'],
                            speaker=event['speakerDetails']['name'],
                            host=event['host']['name'],
                            description=event['description'],
                            event_time=event['dateTime'])

    tokenizer = tiktoken.get_encoding(encoding_name)

    prompt_tokens= tokenizer.encode(prompt)
    num_input_tokens= len(prompt_tokens)
    if num_input_tokens > num_max_input_tokens:
      return ('',prompt,f'Exceeded number of tokens. Got:{num_input_tokens} maximum is {num_max_input_tokens}')

    client = oai.OpenAI(api_key=open_ai_key)
    completion = client.chat.completions.create(model=openai_model,
                                                max_tokens=num_max_tokens,
                                                temperature=temperature,
                                                messages=[{"role": "user", "content": prompt}])
    post= completion.choices[0].message.content
    date_str= event['dateTime']
    post += f'\n\n{format_date(date_str)}'

    speaker_name= event['speakerDetails']['name']

    if len(event['speakerDetails']['description'])>0:
      speaker_bio= social_platform_url=event['speakerDetails']['description']
      post += f'\n\n{speaker_bio}'

    if len(event['speakerDetails']['socialNetworks'])>0:
      social= event['speakerDetails']['socialNetworks'][0]['service']
      social_url= social_platform_url=event['speakerDetails']['socialNetworks'][0]['url']
      post += f'\n{speaker_name} {social} profile: {social_url}'

    post += '\n\n'
    post += gen_ai_guru_description

    return ( post,prompt,'')

  except Exception as exc:
    return ('',prompt,str(exc))

  return ('','','Internal error')

In [22]:
input_meetup_response= gr.Textbox(value="", label="JSon response from Meetup", show_label=True)
input_event_id= gr.Textbox(value="296972829", lines=1, max_lines=1, label="Meetup event ID", show_label=True)
input_temperature= gr.Slider(minimum=0.0, maximum=2.0, value=temperature, label="Temperature", show_label=True)
output_post= gr.Textbox(value="", label="LinkedIn post", show_label=True)
output_prompt= gr.Textbox(value="", label="Prompt", show_label=True)
output_error= gr.Textbox(value="", label="Error", show_label=True)

demo = gr.Interface(fn=generate_linkedin_post,
                    inputs=[input_meetup_response, input_temperature],
                    outputs=[output_post,output_prompt,output_error],
                    allow_flagging='never',
                    description='<center><h1>Create a LinkedIn post from a Meetup event</h1></center>')

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 4.24.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://cc768655be718b50c8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


<b>Example of a generated post:</b>



Get ready to be amazed by the wonders of Azure's AI Symphony in our upcoming Meetup event! 🚀

Join us as Ioannis Kozaris takes the virtual stage to dive deep into the realm of AI orchestration, embeddings, and so much more. From exploring the cutting-edge AI services provided by Microsoft Azure to guiding attendees on how to prototype innovative solutions using online tools like Language Studio, this presentation is packed with valuable hands-on insights.

But that's not all! Carlos Hernandez-Vaquero will be hosting this exciting session, offering a unique opportunity for participants to learn about the nuances of machine learning system design and optimization. Discover how to effectively deploy AI systems in production environments and gain a deeper understanding of the strategic orchestration required for successful implementation in real-world scenarios.

Don't miss out on the chance to expand your knowledge, connect with like-minded professionals, and be part of the growing GenAI Guru community. Mark your calendars for November 16th and join us for an evening filled with groundbreaking discussions, valuable learnings, and the opportunity to be part of a vibrant community dedicated to embracing new technologies. See you there! 👩‍💻👨‍💻

Thursday, November 16, 2023 at 06:30 PM CET

Ioannis Kozaris, a Software Engineer specializing in machine learning systems, has been at the forefront of driving innovation in collaborative systems, data storage, and Natural Language Processing systems at Microsoft. He has notably architected ML orchestration systems and intelligent context-aware ML Agents, elevating user experience and optimizing operational efficiency in production applications. Currently, he is part of a task force at Microsoft, dedicated to pioneering applications in Generative AI, setting the course for the future of AI innovations. He is deeply committed to sharing his passion for ML systems with others, aiming to inspire and engage fellow enthusiasts in this evolving field.
Ioannis Kozaris LINKEDIN profile: https://www.linkedin.com/in/ioannis-kozaris/

Gen AI Guru is an innovative, non-profit organization that stands at the exciting intersection of science and arts, powered by Generative AI.
We recognize the transformative potential of these technologies on the workplace and the broader societal landscape. By joining GenAI Gurus, you don't just join a community; you position yourself in the driving seat of a network of passionate individuals, all committed to shaping the future.